<a href="https://colab.research.google.com/github/IamRam3/dry-fruit-detection-segmentation/blob/main/test_modularity_(dryfruits)_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
from PIL import Image
from pycocotools.coco import COCO
import torch.utils.data
import torchvision
from torchvision.transforms import ToTensor
from pycocotools.cocoeval import COCOeval
import json
import numpy as np
from tqdm import tqdm
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [2]:
!git clone https://github.com/IamRam3/dry-fruit-detection-segmentation.git

Cloning into 'dry-fruit-detection-segmentation'...
remote: Enumerating objects: 345, done.
remote: Counting objects: 100% (345/345), done.
remote: Compressing objects: 100% (320/320), done.
remote: Total 345 (delta 51), reused 306 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (345/345), 24.37 MiB | 16.28 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [3]:
!mv dry-fruit-detection-segmentation dry_fruit_detection_segmentation

In [ ]:
!rm -rf dry_fruit_detection_segmentation/

In [4]:
from dry_fruit_detection_segmentation.src.data_setup import CocoDataset, dataset_dir, CustomDataLoader
from dry_fruit_detection_segmentation.utils import get_transform, convert_to_coco_format, get_detection_model, train, create_video_from_images

In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [6]:
batch_size=4

In [ ]:
rm -rf /content/DryFruits-classification.v1i.coco

In [7]:
!unzip /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-classification.v1i.coco.zip -d /content/DryFruits-classification.v1i.coco

Archive:  /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-classification.v1i.coco.zip
  inflating: /content/DryFruits-classification.v1i.coco/README.dataset.txt  
  inflating: /content/DryFruits-classification.v1i.coco/README.roboflow.txt  
   creating: /content/DryFruits-classification.v1i.coco/test/
 extracting: /content/DryFruits-classification.v1i.coco/test/360_F_1248363446_QWsdjuZlE2uK4sg1gCZSMogHM5d5SDqv_webp.rf.98746b3d6b7cd7d53599adf17435b3ff.jpg  
 extracting: /content/DryFruits-classification.v1i.coco/test/ALMOND_REGULAR_ALMOND_REGULAR_753_jpg.rf.0fc0280f63ea517a12ddacfa2926d7c2.jpg  
 extracting: /content/DryFruits-classification.v1i.coco/test/ALMOND_SANORA_ALMOND_SANORA_292_jpg.rf.ff6e25a988daba132ee36167d5d68c71.jpg  
 extracting: /content/DryFruits-classification.v1i.coco/test/ALMOND_SANORA_ALMOND_SANORA_462_jpg.rf.26ce49555ae7916de41b848804580315.jpg  
 extracting: /content/DryFruits-classification.v1i.coco/test/CASHEW_JUMBO_CASHEW_JUMBO_797

In [ ]:
!unzip /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-Detection.v2i.coco.zip -d /content/DryFruits-Detection.v2i.coco

Archive:  /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-Detection.v2i.coco.zip
  inflating: /content/DryFruits-Detection.v2i.coco/README.dataset.txt  
  inflating: /content/DryFruits-Detection.v2i.coco/README.roboflow.txt  
   creating: /content/DryFruits-Detection.v2i.coco/test/
 extracting: /content/DryFruits-Detection.v2i.coco/test/360_F_1248363446_QWsdjuZlE2uK4sg1gCZSMogHM5d5SDqv_webp.rf.98746b3d6b7cd7d53599adf17435b3ff.jpg  
 extracting: /content/DryFruits-Detection.v2i.coco/test/ALMOND_REGULAR_ALMOND_REGULAR_753_jpg.rf.0fc0280f63ea517a12ddacfa2926d7c2.jpg  
 extracting: /content/DryFruits-Detection.v2i.coco/test/ALMOND_SANORA_ALMOND_SANORA_292_jpg.rf.ff6e25a988daba132ee36167d5d68c71.jpg  
 extracting: /content/DryFruits-Detection.v2i.coco/test/ALMOND_SANORA_ALMOND_SANORA_462_jpg.rf.26ce49555ae7916de41b848804580315.jpg  
 extracting: /content/DryFruits-Detection.v2i.coco/test/CASHEW_JUMBO_CASHEW_JUMBO_797_jpg.rf.2a382bd15579fa6f60f1caf5722564ce.jpg 

In [ ]:
!unzip /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-Segmentation.v1i.coco.zip -d /content/DryFruits-Segmentation.v1i.coco

Archive:  /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-Segmentation.v1i.coco.zip
  inflating: /content/DryFruits-Segmentation.v1i.coco/README.dataset.txt  
  inflating: /content/DryFruits-Segmentation.v1i.coco/README.roboflow.txt  
   creating: /content/DryFruits-Segmentation.v1i.coco/test/
 extracting: /content/DryFruits-Segmentation.v1i.coco/test/360_F_1248363446_QWsdjuZlE2uK4sg1gCZSMogHM5d5SDqv_webp.rf.ee5a89bca40b5620557c06690a160e4c.jpg  
 extracting: /content/DryFruits-Segmentation.v1i.coco/test/ALMOND_REGULAR_ALMOND_REGULAR_806_jpg.rf.de00fdc33a36d4aed2d2b3354c47ed7c.jpg  
 extracting: /content/DryFruits-Segmentation.v1i.coco/test/ALMOND_SANORA_ALMOND_SANORA_292_jpg.rf.089d18d7179d9d70c86f32037cafe374.jpg  
 extracting: /content/DryFruits-Segmentation.v1i.coco/test/ALMOND_SANORA_ALMOND_SANORA_905_jpg.rf.822d8eb52e60f587dfb543e358580f32.jpg  
 extracting: /content/DryFruits-Segmentation.v1i.coco/test/CASHEW_SPECIAL_CASHEW_SPECIAL_552_jpg.rf.62b04a

In [ ]:
!unzip /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-Segmentation.v2i.coco.zip -d /content/DryFruits-Segmentation.v2i.coco

Archive:  /content/dry_fruit_detection_segmentation/datasets/labeled_dataset2/DryFruits-Segmentation.v2i.coco.zip
  inflating: /content/DryFruits-Segmentation.v2i.coco/README.dataset.txt  
  inflating: /content/DryFruits-Segmentation.v2i.coco/README.roboflow.txt  
   creating: /content/DryFruits-Segmentation.v2i.coco/test/
 extracting: /content/DryFruits-Segmentation.v2i.coco/test/360_F_1248363446_QWsdjuZlE2uK4sg1gCZSMogHM5d5SDqv_webp.rf.ee5a89bca40b5620557c06690a160e4c.jpg  
 extracting: /content/DryFruits-Segmentation.v2i.coco/test/ALMOND_REGULAR_ALMOND_REGULAR_806_jpg.rf.de00fdc33a36d4aed2d2b3354c47ed7c.jpg  
 extracting: /content/DryFruits-Segmentation.v2i.coco/test/ALMOND_SANORA_ALMOND_SANORA_292_jpg.rf.089d18d7179d9d70c86f32037cafe374.jpg  
 extracting: /content/DryFruits-Segmentation.v2i.coco/test/ALMOND_SANORA_ALMOND_SANORA_905_jpg.rf.822d8eb52e60f587dfb543e358580f32.jpg  
 extracting: /content/DryFruits-Segmentation.v2i.coco/test/CASHEW_SPECIAL_CASHEW_SPECIAL_552_jpg.rf.62b04a

In [8]:
dataset_path = "/content/DryFruits-classification.v1i.coco"

In [9]:
dataset = dataset_dir(dataset_path)

In [10]:
train_dataset, val_dataset, test_dataset = dataset.create_custom_datasets(ToTensor, segmentation=False)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [11]:
train_data_loader, val_data_loader, test_data_loader = CustomDataLoader(train_dataset, val_dataset, test_dataset, batch_size).create_data_loader()

In [12]:
from dry_fruit_detection_segmentation.utils import FocalLoss, custom_fastrcnn_loss, get_segmentation_model

In [13]:
# extra class; for background
num_classes = len(train_dataset.coco.getCatIds()) + 1
model = get_detection_model(num_classes, mobilenet=True, custom_focal_loss=False)

# move model to the right device
model.to(device)

# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

num_epochs = 100

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_fpn-fb6a3cc7.pth
100%|██████████| 74.2M/74.2M [00:00<00:00, 101MB/s] 


In [14]:
best_map = 0.0
epochs_no_improve = 0
patience = 5  # stop if no improvement for 5 consecutive epochs
save_path = "mobilenet_v1.pth"

In [ ]:
train(num_epochs, train_data_loader, model, val_data_loader, convert_to_coco_format, dataset
, device, optimizer, save_path, patience=5, best_map=0.0, segmentation=False)

Training Epoch 1: 100%|██████████| 18/18 [00:02<00:00,  6.45it/s]


[Epoch 1] Training Loss: 1.9449


Validating: 100%|██████████| 5/5 [00:00<00:00, 12.52it/s]


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
[Epoch 1] Evaluation Metrics:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.318
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.605
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.318
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.548
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.548
 Average Recall     (AR) @[ IoU=0

Training Epoch 2: 100%|██████████| 18/18 [00:02<00:00,  7.64it/s]


...

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.02s).
[Epoch 20] Evaluation Metrics:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.691
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.853
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.853
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.691
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.743
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.743
 Average Recall     (AR) @[ IoU=

Training Epoch 21: 100%|██████████| 18/18 [00:02<00:00,  6.95it/s]


[Epoch 21] Training Loss: 0.3550


Validating: 100%|██████████| 5/5 [00:00<00:00, 11.53it/s]

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.01s).
[Epoch 21] Evaluation Metrics:
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.722
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.886
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.886
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.722
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.790
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.790
 Average Recall     (AR) @[ IoU=

In [ ]:
create_video_from_images(dataset.test_dir, "object_detection_in.mp4", fps=1)

Video saved to object_detection_in.mp4


In [ ]:
model = get_segmentation_model(num_classes, custom_focal_loss=False)
model.load_state_dict(torch.load("segmentation_v1.pth"))
model.to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [ ]:
id_to_name = {
    1: "almond",
    2: "cashew",
    3: "grapes",
    4: "raisin"
}

In [ ]:
example_img = "/360_F_1248363446_QWsdjuZlE2uK4sg1gCZSMogHM5d5SDqv_webp.rf.98746b3d6b7cd7d53599adf17435b3ff.jpg"

In [ ]:
from dry_fruit_detection_segmentation.utils import save_predicted_image, save_predicted_video

In [ ]:
from dry_fruit_detection_segmentation.utils import closest_basic_color_name, detect_shape, save_segmented_image

In [ ]:
save_predicted_image("/content/DryFruits-Detection.v2i.coco/test/ALMOND_REGULAR_ALMOND_REGULAR_753_jpg.rf.0fc0280f63ea517a12ddacfa2926d7c2.jpg", "output4.jpg", model, device, id_to_name)

In [ ]:
save_predicted_video("/content/object_detection_in.mp4", "segmentation_v1.mp4", model, device, id_to_name, threshold=0.4)

In [ ]:
model = get_detection_model(num_classes, mobilenet=True, custom_focal_loss=False)
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
          )
        )
      )
      (2): InvertedResidual(
        (block):

In [ ]:
model.load_state_dict(torch.load("mobilenet_v1.pth"))

# Set to eval mode
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): FrozenBatchNorm2d(16, eps=1e-05)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
            (2): ReLU(inplace=True)
          )
          (1): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d(16, eps=1e-05)
          )
        )
      )
      (2): InvertedResidual(
        (block):

In [ ]:
!pip install torch torchvision onnx tf2onnx onnx-tf tensorflow

INFO: pip is looking at multiple versions of tf2onnx to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer 

In [ ]:
from PIL import Image
import torchvision.transforms as T

image = Image.open("/content/DryFruits-classification.v1i.coco/test/ALMOND_REGULAR_ALMOND_REGULAR_753_jpg.rf.0fc0280f63ea517a12ddacfa2926d7c2.jpg").convert("RGB")
transform = T.ToTensor()
tensor_img = transform(image).unsqueeze(0)  # shape: (1, 3, H, W)

print(tensor_img.shape)

torch.Size([1, 3, 640, 640])


# In put shape - ([1, 3, 640, 640])

In [ ]:
dummy_input = torch.randn(1, 3, 640, 640)

torch.onnx.export(model, dummy_input, "mobilenet_v1.onnx",
                  input_names=['input'], output_names=['output'],
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})

In [ ]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00


In [ ]:
import onnxruntime as ort
import numpy as np

session = ort.InferenceSession("mobilenet_v1.onnx")
outputs = session.run(None, {"input": np.random.randn(1, 3, 640, 640).astype(np.float32)})

print(outputs)  # should print boxes, labels, scores, masks


[array([], shape=(0, 4), dtype=float32), array([], dtype=int64), array([], dtype=float32)]


In [ ]:
import cv2
from torchvision.transforms import functional as F

# Load and preprocess image
image = cv2.imread(dataset.test_dir+example_img)
image = cv2.resize(image, (640, 640))
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Normalize and convert to tensor
image_tensor = F.to_tensor(image_rgb).unsqueeze(0).numpy().astype(np.float32)

outputs = session.run(None, {"input": image_tensor})
print(outputs)


RuntimeException: [ONNXRuntimeError] : 6 : RUNTIME_EXCEPTION : Non-zero status code returned while running Reshape node. Name:'/roi_heads/Reshape_2' Status Message: /onnxruntime_src/onnxruntime/core/providers/cpu/tensor/reshape_helper.h:39 onnxruntime::ReshapeHelper::ReshapeHelper(const onnxruntime::TensorShape&, onnxruntime::TensorShapeVector&, bool) size != 0 && (input_shape_size % size) == 0 was false. The input tensor cannot be reshaped to the requested shape. Input shape:{82,23}, requested shape:{-1,4}


In [ ]:
import torch
import torchvision.transforms as T
from PIL import Image
import numpy as np

# Dummy input image
dummy_image = torch.rand(1, 3, 640, 640)  # [B, C, H, W]
torch.onnx.export(
    model_frcnn,
    (dummy_image,),
    "fasterrcnn.onnx",
    input_names=["input"],
    output_names=["boxes", "labels", "scores"],
    dynamic_axes={"input": {0: "batch_size"}},
    opset_version=11,
)
print("[✓] Exported Faster R-CNN to ONNX")


In [ ]:
import torch.nn as nn

class MaskRCNNWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        outputs = self.model(x)
        boxes = outputs[0]["boxes"]
        labels = outputs[0]["labels"]
        scores = outputs[0]["scores"]
        masks = outputs[0]["masks"]
        return boxes, labels, scores, masks


In [ ]:
import torch.nn as nn

class FasterRCNNWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        outputs = self.model(x)
        boxes = outputs[0]["boxes"]
        labels = outputs[0]["labels"]
        scores = outputs[0]["scores"]
        return boxes, labels, scores


In [ ]:
wrapped_model = FasterRCNNWrapper(model)
wrapped_model.eval()

torch.onnx.export(
    wrapped_model,
    dummy_input,
    "fasterrcnn_model.onnx",
    export_params=True,
    opset_version=11,
    input_names=["input"],
    output_names=["boxes", "labels", "scores"],
    dynamic_axes={
        "input": {0: "batch_size"},
        "boxes": {0: "batch_size"},
        "labels": {0: "batch_size"},
        "scores": {0: "batch_size"},
    }
)


In [ ]:
wrapped_model = MaskRCNNWrapper(model)
wrapped_model.eval()

dummy_input = torch.randn(1, 3, 512, 512)  # match your actual input shape

torch.onnx.export(
    wrapped_model,
    dummy_input,
    "maskrcnn_model.onnx",
    export_params=True,
    opset_version=11,
    input_names=["input"],
    output_names=["boxes", "labels", "scores", "masks"],
    dynamic_axes={
        "input": {0: "batch_size"},
        "boxes": {0: "batch_size"},
        "labels": {0: "batch_size"},
        "scores": {0: "batch_size"},
        "masks": {0: "batch_size"},
    }
)


validation

In [ ]:
import torch
import torchvision.models as models

mobilenet = models.mobilenet_v2(pretrained=True)
mobilenet.eval()

dummy_input = torch.randn(1, 3, 224, 224)

torch.onnx.export(mobilenet, dummy_input, "mobilenet_v2.onnx",
                  input_names=['input'], output_names=['output'],
                  dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}})


In [ ]:
pip install onnx-tf tensorflow

In [ ]:
# Convert ONNX to TensorFlow
from onnx_tf.backend import prepare
import onnx

onnx_model = onnx.load("mobilenet_v2.onnx")
tf_rep = prepare(onnx_model)
tf_rep.export_graph("mobilenet_tf")

# Convert TensorFlow to TFLite
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("mobilenet_tf")
tflite_model = converter.convert()

with open("mobilenet.tflite", "wb") as f:
    f.write(tflite_model)
